In [1]:
import os
import itertools
from collections import defaultdict

In [2]:
import pandas as pd
from functools import reduce
import operator

In [3]:
# Adding module to sys path
import sys
sys.path.append("/home/cfourrie/documents/software/public/CopperMT/")
# RNN imports
import pipeline
import torch, numpy as np
from fairseq import checkpoint_utils, data, options, tasks
from pipeline.neural_translation.multilingual_rnns.multilingual_rnn import MultilingualRNNModel

In [4]:
raw_data_path = "inputs/raw_data/" 
split_data_path = "inputs/split_data/"

In [5]:
folders = ["abrahammonpa", "allenbai", "backstromnorthernpakistan", "castrosui", "davletshinaztecan", 
           "felekesemitic", "hantganbangime", "hattorijaponic", "listsamplesize", "mannburmish"]
splits = ["0.10", "0.20", "0.30", "0.40", "0.50"]
train_name = "training"
test_in_name = "test"
test_out_name = "solutions"
save_as = "results"

In [30]:
conversion_needed = True
preprocessing_needed = False


In [7]:
models_dir = "/home/cfourrie/documents/software/public/CopperMT/workspace"

## Convert data to usual format for our software

In [33]:
# If you do not want to share embeddings, removed shared_ from the folder path
if conversion_needed:
    for folder in folders:
        for split in splits:
            try: os.makedirs(os.path.join(split_data_path, f"shared_{folder}", split))
            except FileExistsError: pass
            # Training
            cur_data = pd.read_csv(os.path.join(raw_data_path, folder, f"training-{split}.tsv"), sep="\t")
            cur_languages = [c for c in cur_data.columns if c != 'COGID']
            for l1, l2 in itertools.combinations(cur_languages, 2):
                for name in [f"{l1}-{l2}", f"{l2}-{l1}"]:
                    with open(os.path.join(split_data_path, f"shared_{folder}", split, f"train.{name}.{l1}"), "w+") as f1, \
                         open(os.path.join(split_data_path, f"shared_{folder}", split, f"train.{name}.{l2}"), "w+") as f2:
                        for ix, row in cur_data[[l1, l2]].dropna().iterrows():
                            f1.write(row[l1] + "\n")
                            f2.write(row[l2] + "\n")
            for l in cur_languages:
                with open(os.path.join(split_data_path, f"shared_{folder}", split, f"train.{l}-{l}.{l}"), "w+") as f:
                    for ix, row in cur_data[[l]].dropna().iterrows():
                        f.write(row[l] + "\n")

## Generating configuration files contents

In [9]:
if preprocessing_needed:
    folder = folders[9]
    cur_data = pd.read_csv(os.path.join(raw_data_path, folder, f"cognates.tsv"), sep="\t")
    cur_languages = [c for c in cur_data.columns if c != 'COGID']

    print('PROJ_DIR="/home/cfourrie/documents/software/CopperMT"')
    print('MOSES_DIR="${PROJ_DIR}/submodules"')
    print()
    print('WK_DIR="${PROJ_DIR}/workspace/' + folder + '"')
    print('INPUTS_DIR="${PROJ_DIR}/inputs"')
    print()
    print(f'DATA_NAME="{folder}"')

    print(f'langs_bi="{",".join("-".join(l) for l in itertools.product(cur_languages, cur_languages))}"')
    print(f'langs="{",".join(cur_languages)}"')


Step: 1h30 - In the pipeline folder, execute:

`
bash data_preprocess.sh parameters_abrahammonpa.cfg
bash data_preprocess.sh parameters_allenbai.cfg
bash data_preprocess.sh parameters_backstromnorthernpakistan.cfg
bash data_preprocess.sh parameters_castrosui.cfg
bash data_preprocess.sh parameters_davletshinaztecan.cfg
bash data_preprocess.sh parameters_felekesemitic.cfg
bash data_preprocess.sh parameters_hantganbangime.cfg
bash data_preprocess.sh parameters_hattorijaponic.cfg
bash data_preprocess.sh parameters_listsamplesize.cfg
bash data_preprocess.sh parameters_mannburmish.cfg
`

## Training models

In the pipeline folder, train bilingual neural models with:

`
bash main_nmt_bilingual_full.sh parameters_abrahammonpa.cfg
bash main_nmt_bilingual_full.sh parameters_allenbai.cfg
bash main_nmt_bilingual_full.sh parameters_backstromnorthernpakistan.cfg
bash main_nmt_bilingual_full.sh parameters_castrosui.cfg
bash main_nmt_bilingual_full.sh parameters_davletshinaztecan.cfg
bash main_nmt_bilingual_full.sh parameters_felekesemitic.cfg
bash main_nmt_bilingual_full.sh parameters_hantganbangime.cfg
bash main_nmt_bilingual_full.sh parameters_hattorijaponic.cfg
bash main_nmt_bilingual_full.sh parameters_listsamplesize.cfg
bash main_nmt_bilingual_full.sh parameters_mannburmish.cfg
`

In the pipeline folder, train multilingual neural models with:

`
bash main_nmt_multilingual_full.sh parameters_abrahammonpa.cfg
bash main_nmt_multilingual_full.sh parameters_allenbai.cfg
bash main_nmt_multilingual_full.sh parameters_backstromnorthernpakistan.cfg
bash main_nmt_multilingual_full.sh parameters_castrosui.cfg
bash main_nmt_multilingual_full.sh parameters_davletshinaztecan.cfg
bash main_nmt_multilingual_full.sh parameters_felekesemitic.cfg
bash main_nmt_multilingual_full.sh parameters_hantganbangime.cfg
bash main_nmt_multilingual_full.sh parameters_hattorijaponic.cfg
bash main_nmt_multilingual_full.sh parameters_listsamplesize.cfg
bash main_nmt_multilingual_full.sh parameters_mannburmish.cfg
`


In the pipeline folder, train bilingual statistical models with:

`
bash main_smt_full.sh parameters_abrahammonpa.cfg
bash main_smt_full.sh parameters_allenbai.cfg
bash main_smt_full.sh parameters_backstromnorthernpakistan.cfg
bash main_smt_full.sh parameters_castrosui.cfg
bash main_smt_full.sh parameters_davletshinaztecan.cfg
bash main_smt_full.sh parameters_felekesemitic.cfg
bash main_smt_full.sh parameters_hantganbangime.cfg
bash main_smt_full.sh parameters_hattorijaponic.cfg
bash main_smt_full.sh parameters_listsamplesize.cfg
bash main_smt_full.sh parameters_mannburmish.cfg
`

# Model utils

In [10]:
cur_data = folders[0]
split = "0.10"

In [11]:
all_models = {model: {} for model in ["BiNMT", "MNMT"]}
lang2indexed_phones_all_r = {model: defaultdict(dict) for model in ["BiNMT", "MNMT"]}
lang2indexed_phones_all = {model: defaultdict(dict) for model in ["BiNMT", "MNMT"]}

df = pd.read_csv(os.path.join(raw_data_path, cur_data, f"cognates.tsv"), sep="\t")
langs = [c for c in df.columns if c != 'COGID']

for l_out in langs:
    for l_in in langs:
        if l_out == l_in: continue
        lpair = f"{l_in}-{l_out}"
        for model, path in [
            ("BiNMT", f"{models_dir}/{cur_data}/bilingual/{lpair}"), 
            ("MNMT", f"{models_dir}/{cur_data}/multilingual")
        ]:
            path = f"{path}/{split}/checkpoints/checkpoint_last.pt"
            ensemble, _, t = checkpoint_utils.load_model_ensemble_and_task([path]);
            all_models[model][lpair] = ensemble[0].models[lpair]
            all_models[model][lpair].eval()
            for lang, vals in t.dicts.items():
                lang2indexed_phones_all[model][lpair][lang] = vals.indices
                lang2indexed_phones_all_r[model][lpair][lang] = {v:k for k, v in vals.indices.items()}

Namespace(adam_betas='(0.9, 0.999)', adam_eps=1e-08, adaptive_softmax_cutoff='10000,50000,200000', all_gather_list_size=16384, arch='multilingual_bigru', attention_type='luong-general', batch_size=10, batch_size_valid=10, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=0.0, cpu=False, criterion='cross_entropy', curriculum=0, data='/home/cfourrie/documents/software/public/CopperMT/workspace/abrahammonpa/bilingual/data/0.10/data-bin/', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_dropout_in=-1, decoder_dropout_out=-1, decoder_embed_dim=24, decoder_embed_path=None, decoder_embeddings_sharing=[['MonpaBalemu']], decoder_freeze_embed=False, decoder_hidden_size=72, decoder_langtok=False, decoder_layers=2, decoder_out_embed_dim=512, decoders_sharing=[['MonpaBalemu']], device_id=0, disable_validation=True, distributed_backend='nccl', distributed_init_method=None, dist

Namespace(adam_betas='(0.9, 0.999)', adam_eps=1e-08, adaptive_softmax_cutoff='10000,50000,200000', all_gather_list_size=16384, arch='multilingual_bigru', attention_type='luong-general', batch_size=10, batch_size_valid=10, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=0.0, cpu=False, criterion='cross_entropy', curriculum=0, data='/home/cfourrie/documents/software/public/CopperMT/workspace/abrahammonpa/bilingual/data/0.10/data-bin/', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_dropout_in=-1, decoder_dropout_out=-1, decoder_embed_dim=24, decoder_embed_path=None, decoder_embeddings_sharing=[['MonpaBalemu']], decoder_freeze_embed=False, decoder_hidden_size=72, decoder_langtok=False, decoder_layers=2, decoder_out_embed_dim=512, decoders_sharing=[['MonpaBalemu']], device_id=0, disable_validation=True, distributed_backend='nccl', distributed_init_method=None, dist

Namespace(adam_betas='(0.9, 0.999)', adam_eps=1e-08, adaptive_softmax_cutoff='10000,50000,200000', all_gather_list_size=16384, arch='multilingual_bigru', attention_type='luong-general', batch_size=10, batch_size_valid=10, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=0.0, cpu=False, criterion='cross_entropy', curriculum=0, data='/home/cfourrie/documents/software/public/CopperMT/workspace/abrahammonpa/bilingual/data/0.10/data-bin/', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_dropout_in=-1, decoder_dropout_out=-1, decoder_embed_dim=24, decoder_embed_path=None, decoder_embeddings_sharing=[['MonpaBalemu']], decoder_freeze_embed=False, decoder_hidden_size=72, decoder_langtok=False, decoder_layers=2, decoder_out_embed_dim=512, decoders_sharing=[['MonpaBalemu']], device_id=0, disable_validation=True, distributed_backend='nccl', distributed_init_method=None, dist

Namespace(adam_betas='(0.9, 0.999)', adam_eps=1e-08, adaptive_softmax_cutoff='10000,50000,200000', all_gather_list_size=16384, arch='multilingual_bigru', attention_type='luong-general', batch_size=10, batch_size_valid=10, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=0.0, cpu=False, criterion='cross_entropy', curriculum=0, data='/home/cfourrie/documents/software/public/CopperMT/workspace/abrahammonpa/bilingual/data/0.10/data-bin/', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_dropout_in=-1, decoder_dropout_out=-1, decoder_embed_dim=24, decoder_embed_path=None, decoder_embeddings_sharing=[['MonpaBalemu']], decoder_freeze_embed=False, decoder_hidden_size=72, decoder_langtok=False, decoder_layers=2, decoder_out_embed_dim=512, decoders_sharing=[['MonpaBalemu']], device_id=0, disable_validation=True, distributed_backend='nccl', distributed_init_method=None, dist

Namespace(adam_betas='(0.9, 0.999)', adam_eps=1e-08, adaptive_softmax_cutoff='10000,50000,200000', all_gather_list_size=16384, arch='multilingual_bigru', attention_type='luong-general', batch_size=10, batch_size_valid=10, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=0.0, cpu=False, criterion='cross_entropy', curriculum=0, data='/home/cfourrie/documents/software/public/CopperMT/workspace/abrahammonpa/bilingual/data/0.10/data-bin/', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_dropout_in=-1, decoder_dropout_out=-1, decoder_embed_dim=24, decoder_embed_path=None, decoder_embeddings_sharing=[['MonpaBalemu']], decoder_freeze_embed=False, decoder_hidden_size=72, decoder_langtok=False, decoder_layers=2, decoder_out_embed_dim=512, decoders_sharing=[['MonpaBalemu']], device_id=0, disable_validation=True, distributed_backend='nccl', distributed_init_method=None, dist

...

Namespace(adam_betas='(0.9, 0.999)', adam_eps=1e-08, adaptive_softmax_cutoff='10000,50000,200000', all_gather_list_size=16384, arch='multilingual_bigru', attention_type='luong-general', batch_size=10, batch_size_valid=10, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=0.0, cpu=False, criterion='cross_entropy', curriculum=0, data='/home/cfourrie/documents/software/public/CopperMT/workspace/abrahammonpa/bilingual/data/0.10/data-bin/', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_dropout_in=-1, decoder_dropout_out=-1, decoder_embed_dim=24, decoder_embed_path=None, decoder_embeddings_sharing=[['MonpaTomko']], decoder_freeze_embed=False, decoder_hidden_size=72, decoder_langtok=False, decoder_layers=2, decoder_out_embed_dim=512, decoders_sharing=[['MonpaTomko']], device_id=0, disable_validation=True, distributed_backend='nccl', distributed_init_method=None, distri

Namespace(adam_betas='(0.9, 0.999)', adam_eps=1e-08, adaptive_softmax_cutoff='10000,50000,200000', all_gather_list_size=16384, arch='multilingual_bigru', attention_type='luong-general', batch_size=10, batch_size_valid=10, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=0.0, cpu=False, criterion='cross_entropy', curriculum=0, data='/home/cfourrie/documents/software/public/CopperMT/workspace/abrahammonpa/bilingual/data/0.10/data-bin/', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_dropout_in=-1, decoder_dropout_out=-1, decoder_embed_dim=24, decoder_embed_path=None, decoder_embeddings_sharing=[['MonpaTomko']], decoder_freeze_embed=False, decoder_hidden_size=72, decoder_langtok=False, decoder_layers=2, decoder_out_embed_dim=512, decoders_sharing=[['MonpaTomko']], device_id=0, disable_validation=True, distributed_backend='nccl', distributed_init_method=None, distri

Namespace(adam_betas='(0.9, 0.999)', adam_eps=1e-08, adaptive_softmax_cutoff='10000,50000,200000', all_gather_list_size=16384, arch='multilingual_bigru', attention_type='luong-general', batch_size=10, batch_size_valid=10, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=0.0, cpu=False, criterion='cross_entropy', curriculum=0, data='/home/cfourrie/documents/software/public/CopperMT/workspace/abrahammonpa/bilingual/data/0.10/data-bin/', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_dropout_in=-1, decoder_dropout_out=-1, decoder_embed_dim=24, decoder_embed_path=None, decoder_embeddings_sharing=[['MonpaTomko']], decoder_freeze_embed=False, decoder_hidden_size=72, decoder_langtok=False, decoder_layers=2, decoder_out_embed_dim=512, decoders_sharing=[['MonpaTomko']], device_id=0, disable_validation=True, distributed_backend='nccl', distributed_init_method=None, distri

Namespace(adam_betas='(0.9, 0.999)', adam_eps=1e-08, adaptive_softmax_cutoff='10000,50000,200000', all_gather_list_size=16384, arch='multilingual_bigru', attention_type='luong-general', batch_size=10, batch_size_valid=10, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=0.0, cpu=False, criterion='cross_entropy', curriculum=0, data='/home/cfourrie/documents/software/public/CopperMT/workspace/abrahammonpa/bilingual/data/0.10/data-bin/', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_dropout_in=-1, decoder_dropout_out=-1, decoder_embed_dim=24, decoder_embed_path=None, decoder_embeddings_sharing=[['MonpaTomko']], decoder_freeze_embed=False, decoder_hidden_size=72, decoder_langtok=False, decoder_layers=2, decoder_out_embed_dim=512, decoders_sharing=[['MonpaTomko']], device_id=0, disable_validation=True, distributed_backend='nccl', distributed_init_method=None, distri

Namespace(adam_betas='(0.9, 0.999)', adam_eps=1e-08, adaptive_softmax_cutoff='10000,50000,200000', all_gather_list_size=16384, arch='multilingual_bigru', attention_type='luong-general', batch_size=10, batch_size_valid=10, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=0.0, cpu=False, criterion='cross_entropy', curriculum=0, data='/home/cfourrie/documents/software/public/CopperMT/workspace/abrahammonpa/bilingual/data/0.10/data-bin/', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_dropout_in=-1, decoder_dropout_out=-1, decoder_embed_dim=24, decoder_embed_path=None, decoder_embeddings_sharing=[['MonpaTomko']], decoder_freeze_embed=False, decoder_hidden_size=72, decoder_langtok=False, decoder_layers=2, decoder_out_embed_dim=512, decoders_sharing=[['MonpaTomko']], device_id=0, disable_validation=True, distributed_backend='nccl', distributed_init_method=None, distri

In [12]:
def product_list(l, ix = -1):
    """ l the list for which we want the element product, ix the index of the item of interests in the sublists"""
    if ix == -1:
        return reduce(operator.mul, l, 1)
    return reduce(operator.mul, (i[0] for i in l), 1)

In [22]:
def get_n_best_preds_with_model(model, in_lang, out_lang, test_values, n_best):
    softmax = torch.nn.Softmax(0)

    lpair = f"{in_lang}-{out_lang}"
    
    cur_model = all_models[model][lpair]
    pda = lang2indexed_phones_all[model][lpair][in_lang]
    pdar = lang2indexed_phones_all_r[model][lpair][out_lang]
    
    try:
        rev_sample = [pda[cur_char] for cur_char in test_values]
    except KeyError:
        return {}
        
    batch = torch.tensor([rev_sample])
    len_samples = torch.tensor([s.shape[0] for s in batch]).int()
    full_enc_output = cur_model.encoder(batch, len_samples)

    # Create previous tokens (2 followed by ones for a word)
    len_test_values = full_enc_output[0].shape[1]    
    prev_tokens = torch.ones(len_test_values).int()
    prev_tokens = prev_tokens.unsqueeze(0)
    
    output, attn = cur_model.decoder(prev_tokens, full_enc_output)
    
    output = output[0] #first item of a batch of one
    chars = []
    for ix, c in enumerate(output):
        chars.append([])
        values, topk = softmax(c).topk(n_best)
        for n in range(n_best):
            chars[ix].append((float(values[n]), int(topk[n])))
    
    all_results = list(itertools.product(*chars))
    sorted_results = sorted(all_results, key=lambda x: product_list(x, 0), reverse=True)
    
    result = {}

    for n in range(n_best):
        key = [pdar[int(i[1])] for i in sorted_results[n]] + ["</s>"]
        end_ix = key.index("</s>")
        value = product_list(sorted_results[n], ix=0)
        if value * 100 > 1:
            result[" ".join(key[:end_ix])] = value*100
        
    return result

In [57]:
row_dict

{'MonpaBalemu': 'l a m',
 'MonpaDirang': 'l a m',
 'MonpaDirangDum': 'l a m',
 'MonpaKalaktang': 'l a m',
 'MonpaNamsu': 'l a m',
 'MonpaSangti': 'l a m',
 'MonpaTembang': 'l a m',
 'MonpaTomko': '?'}

In [68]:
batch = [row_dict["MonpaBalemu"]]
print(batch)
gen = t.build_generator(all_models[model]["MonpaBalemu-MonpaTomko"], {})
print(gen)
t.args.source_lang = "MonpaBalemu"
t.args.target_lang = "MonpaTomko"
dataset = t.build_dataset_for_inference(batch, torch.tensor([len(s) for s in batch]).int())
t.inference_step(gen)
print(dataset.datasets)

['l a m']


TypeError: object of type 'FairseqEncoderDecoderModel' has no len()

In [23]:
test_df = pd.read_csv(os.path.join(raw_data_path, cur_data, f"test-{split}.tsv"), sep="\t")
final_results = defaultdict(list)
for ix, (_, row) in enumerate(test_df.iterrows()):
    row_dict = dict(row)
    final_results["COGID"].append(row_dict.pop("COGID"))
    lang_out = [k for k, v in row_dict.items() if v == "?"][0]
    row_results = {}
    # Compute all predictions
    for lang, val in row_dict.items():
        if lang == lang_out: continue
        row_results[lang] = get_n_best_preds_with_model("MNMT", lang, lang_out, val.split(" "), 10)
        final_results[lang].append("")
        
    # Compute best prediction
    predictions_scores = defaultdict(int)
    predictions_counts = defaultdict(int)
    for lang_res in row_results.values():
        for pred, score in lang_res.items():
            predictions_scores[pred] += score
            predictions_counts[pred] += 1
    best_prediction = [k for k, v in predictions_scores.items() if v == max(predictions_scores.values())]
    if best_prediction:
        best_prediction = best_prediction[0]
    else:
        best_prediction = ""
    # Save
    final_results[lang_out].append(best_prediction)


In [29]:
with open(os.path.join(raw_data_path, cur_data, f"mypredictions-{split}.tsv"), "w+") as f:
    f.write("COGID\t" + "\t".join(langs) + "\n")
    for ix in range(len(final_results["COGID"])):
        f.write("\t".join([final_results[label][ix] if final_results[label][ix] else "" for label in ["COGID"] + langs]) + "\n")